In [1]:
import pandas as pd
import numpy as np
import uuid 

/Users/shashanksingh/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/shashanksingh/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:61: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


!pip install --upgrade pandas numpy

In [2]:
try:
    fuel_df = pd.read_csv('fuel_prices.csv')
    fuel_df['Date'] = pd.to_datetime(fuel_df['Date'])
except FileNotFoundError:
    print("Error: 'fuel_prices_historical_all_pages.csv' not found.")
    print("Please run the previous script first.")
    exit()

In [3]:
VEHICLE_SPECS = {
    'Motorcycle': {'base_mileage_km_per_l': 45.0, 'fuel_type': 'Petrol'},
    'Van': {'base_mileage_km_per_l': 12.0, 'fuel_type': 'Diesel'},
    'Truck': {'base_mileage_km_per_l': 5.0, 'fuel_type': 'Diesel'}
}

ROUTES = ['Route_A', 'Route_B', 'Route_C']
N_TRIPS_PER_DAY_CITY = 8

In [4]:
all_trips = []
for row in fuel_df.itertuples():
    current_date = row.Date
    current_city = row.City
    petrol_price = row.Petrol_Price
    diesel_price = row.Diesel_Price
    
    if pd.isna(petrol_price) or pd.isna(diesel_price):
        continue
        
    for _ in range(N_TRIPS_PER_DAY_CITY):
        vehicle_type = np.random.choice(['Motorcycle', 'Van', 'Truck'], p=[0.2, 0.5, 0.3])
        specs = VEHICLE_SPECS[vehicle_type]
        
        fuel_type = specs['fuel_type']
        price_per_liter = petrol_price if fuel_type == 'Petrol' else diesel_price
        
        distance_km = round(np.random.uniform(15,3000.0), 2)
        load_weight_kg = 0
        if vehicle_type == 'Van':
            load_weight_kg = round(np.random.uniform(50.0, 800.00), 2)
        elif vehicle_type == 'Truck':
            load_weight_kg = round(np.random.uniform(1000.0, 15000.0), 2)
        mileage_penalty = (load_weight_kg / 1000) * 0.1 
        current_mileage = specs['base_mileage_km_per_l'] - mileage_penalty
        
        fuel_consumed_liters = distance_km / current_mileage
        total_fuel_cost = round(fuel_consumed_liters * price_per_liter, 2)
        
        all_trips.append({
            'Trip_ID': str(uuid.uuid4()),
            'Date': current_date,
            'City': current_city,
            'Vehicle_Type': vehicle_type,
            'Fuel_Type': fuel_type,
            'Route': np.random.choice(ROUTES),
            'Distance_km': distance_km,
            'Load_Weight_kg': load_weight_kg,
            'Total_Fuel_Cost': total_fuel_cost
        })

transport_df = pd.DataFrame(all_trips)

In [5]:
print("\nDataFrame Info:")
transport_df.info()
transport_df.head()


DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96544 entries, 0 to 96543
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Trip_ID          96544 non-null  object        
 1   Date             96544 non-null  datetime64[ns]
 2   City             96544 non-null  object        
 3   Vehicle_Type     96544 non-null  object        
 4   Fuel_Type        96544 non-null  object        
 5   Route            96544 non-null  object        
 6   Distance_km      96544 non-null  float64       
 7   Load_Weight_kg   96544 non-null  float64       
 8   Total_Fuel_Cost  96544 non-null  float64       
dtypes: datetime64[ns](1), float64(3), object(5)
memory usage: 6.6+ MB


,Trip_ID,Date,City,Vehicle_Type,Fuel_Type,Route,Distance_km,Load_Weight_kg,Total_Fuel_Cost
0,c764d41f-9521-4bfc-8c07-2c5fffc8e21b,2025-10-22,Chennai,Truck,Diesel,Route_C,2532.43,14274.99,65492.27
1,dc63e782-8978-4c93-8986-7dd5a9a6a5ec,2025-10-22,Chennai,Truck,Diesel,Route_B,2795.18,7538.01,60818.32
2,02533cdd-17ac-40b7-9b0c-f618ecc4c086,2025-10-22,Chennai,Truck,Diesel,Route_B,2934.21,12934.98,73139.49
3,18bcafd1-56b3-4c1a-9503-6824ed12f37d,2025-10-22,Chennai,Motorcycle,Petrol,Route_C,1964.62,0.00,4400.75
4,b53d379b-9a1c-4aa7-9183-57c6e8c11bc6,2025-10-22,Chennai,Van,Diesel,Route_C,2739.38,706.93,21215.93


In [6]:
transport_df.to_csv("Transport Data.csv", index = False)